In [1]:
from sklearn.decomposition import PCA #PCA算法
from sklearn.ensemble import RandomForestClassifier as RFC #随机森林分类模型
from sklearn.model_selection import cross_val_score #交叉验证
from sklearn.neighbors import KNeighborsClassifier as KNN #KNN算法
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorcircuit as tc
from functools import partial
from sklearn.model_selection import train_test_split
K = tc.set_backend("tensorflow")
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train ,x_test= x_train[..., np.newaxis] / 255.0,  x_test[..., np.newaxis] / 255.0
x_train = tf.image.pad_to_bounding_box(x_train, 2, 2, 32, 32)
x_test = tf.image.pad_to_bounding_box(x_test, 2, 2, 32, 32)

batched_ae = K.vmap(tc.templates.dataset.amplitude_encoding, vectorized_argnums=0)
x_train_q = batched_ae(x_train, 10)
x_test_q = batched_ae(x_test, 10)
y_train = tf.one_hot(y_train,depth=10,on_value=1,off_value=0)  
y_test = tf.one_hot(y_test,depth=10,on_value=1,off_value=0)  

n = 10
blocks = 3
classes=10

def unitary(c,weights,wire1,wire2):
    c.ry(wire1,theta=weights[0])
    c.ry(wire2,theta=weights[1])
    c.cnot(wire1,wire2)

def qml(x, weights):
    output = []
    for k in range(classes):
        c = tc.Circuit(n, inputs=x)
        for j in range(blocks):
            for i in range(n-1):
                unitary(c,weights[k, j, i, :], i, i+1)                
        exp = K.real((K.sum(c.state())))
        output.append(exp)
    outputs = K.stack(output)
    outputs = K.reshape(outputs, [-1])
    return K.softmax(outputs)

qml_layer = tc.keras.QuantumLayer(qml, weights_shape=[classes,blocks, n-1, 2])
model = tf.keras.Sequential([qml_layer])
model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['AUC','accuracy','Recall','Precision'],
)

model.fit(x_train_q, y_train, batch_size=64, epochs=15, validation_data=(x_test_q,y_test))

Epoch 1/15
938/938 [==============================] - 1334s 1s/step - loss: 1.0133 - auc: 0.9549 - accuracy: 0.7453 - recall: 0.4309 - precision: 0.9385 - val_loss: 0.6468 - val_auc: 0.9801 - val_accuracy: 0.8368 - val_recall: 0.6978 - val_precision: 0.9361
Epoch 2/15
938/938 [==============================] - 785s 837ms/step - loss: 0.6029 - auc: 0.9806 - accuracy: 0.8402 - recall: 0.7316 - precision: 0.9274 - val_loss: 0.5296 - val_auc: 0.9846 - val_accuracy: 0.8587 - val_recall: 0.7734 - val_precision: 0.9301
Epoch 3/15
938/938 [==============================] - 787s 840ms/step - loss: 0.5326 - auc: 0.9836 - accuracy: 0.8557 - recall: 0.7758 - precision: 0.9243 - val_loss: 0.4869 - val_auc: 0.9859 - val_accuracy: 0.8671 - val_recall: 0.8006 - val_precision: 0.9243
Epoch 4/15
938/938 [==============================] - 788s 840ms/step - loss: 0.5010 - auc: 0.9848 - accuracy: 0.8626 - recall: 0.7955 - precision: 0.9234 - val_loss: 0.4626 - val_auc: 0.9867 - val_accuracy: 0.8739 - val_r

In [3]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train ,x_test= x_train[..., np.newaxis] / 255.0,  x_test[..., np.newaxis] / 255.0
x_train = tf.image.pad_to_bounding_box(x_train, 2, 2, 32, 32)
x_test = tf.image.pad_to_bounding_box(x_test, 2, 2, 32, 32)

batched_ae = K.vmap(tc.templates.dataset.amplitude_encoding, vectorized_argnums=0)
x_train_q = batched_ae(x_train, 10)
x_test_q = batched_ae(x_test, 10)
y_train = tf.one_hot(y_train,depth=10,on_value=1,off_value=0)  
y_test = tf.one_hot(y_test,depth=10,on_value=1,off_value=0)  

n = 10
blocks = 5
classes=10

def unitary(c,weights,wire1,wire2):
    c.ry(wire1,theta=weights[0])
    c.ry(wire2,theta=weights[1])
    c.cnot(wire1,wire2)

def qml(x, weights):
    output = []
    for k in range(classes):
        c = tc.Circuit(n, inputs=x)
        for j in range(blocks):
            for i in range(n-1):
                unitary(c,weights[k, j, i, :], i, i+1)                
        exp = K.real((K.sum(c.state())))
        output.append(exp)
    outputs = K.stack(output)
    outputs = K.reshape(outputs, [-1])
    return K.softmax(outputs)

qml_layer = tc.keras.QuantumLayer(qml, weights_shape=[classes,blocks, n-1, 2])
model = tf.keras.Sequential([qml_layer])
model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['AUC','accuracy','Recall','Precision'],
)

model.fit(x_train_q, y_train, batch_size=64, epochs=15, validation_data=(x_test_q,y_test))

Epoch 1/15
938/938 [==============================] - 2208s 2s/step - loss: 0.8693 - auc: 0.9636 - accuracy: 0.7780 - recall: 0.5490 - precision: 0.9331 - val_loss: 0.5394 - val_auc: 0.9843 - val_accuracy: 0.8600 - val_recall: 0.7719 - val_precision: 0.9321
Epoch 2/15
938/938 [==============================] - 1874s 2s/step - loss: 0.5065 - auc: 0.9850 - accuracy: 0.8653 - recall: 0.7925 - precision: 0.9289 - val_loss: 0.4442 - val_auc: 0.9882 - val_accuracy: 0.8774 - val_recall: 0.8226 - val_precision: 0.9312
Epoch 3/15
938/938 [==============================] - 1873s 2s/step - loss: 0.4482 - auc: 0.9873 - accuracy: 0.8784 - recall: 0.8249 - precision: 0.9305 - val_loss: 0.4084 - val_auc: 0.9894 - val_accuracy: 0.8891 - val_recall: 0.8457 - val_precision: 0.9338
Epoch 4/15
938/938 [==============================] - 1740s 2s/step - loss: 0.4229 - auc: 0.9883 - accuracy: 0.8846 - recall: 0.8377 - precision: 0.9316 - val_loss: 0.3905 - val_auc: 0.9900 - val_accuracy: 0.8933 - val_recall:

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train ,x_test= x_train[..., np.newaxis] / 255.0,  x_test[..., np.newaxis] / 255.0
x_train = tf.image.pad_to_bounding_box(x_train, 2, 2, 32, 32)
x_test = tf.image.pad_to_bounding_box(x_test, 2, 2, 32, 32)

batched_ae = K.vmap(tc.templates.dataset.amplitude_encoding, vectorized_argnums=0)
x_train_q = batched_ae(x_train, 10)
x_test_q = batched_ae(x_test, 10)
y_train = tf.one_hot(y_train,depth=10,on_value=1,off_value=0)  
y_test = tf.one_hot(y_test,depth=10,on_value=1,off_value=0)  

n = 10
blocks = 7
classes=10

def unitary(c,weights,wire1,wire2):
    c.ry(wire1,theta=weights[0])
    c.ry(wire2,theta=weights[1])
    c.cnot(wire1,wire2)

def qml(x, weights):
    output = []
    for k in range(classes):
        c = tc.Circuit(n, inputs=x)
        for j in range(blocks):
            for i in range(n-1):
                unitary(c,weights[k, j, i, :], i, i+1)                
        exp = K.real((K.sum(c.state())))
        output.append(exp)
    outputs = K.stack(output)
    outputs = K.reshape(outputs, [-1])
    return K.softmax(outputs)

qml_layer = tc.keras.QuantumLayer(qml, weights_shape=[classes,blocks, n-1, 2])
model = tf.keras.Sequential([qml_layer])
model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['AUC','accuracy','Recall','Precision'],
)

model.fit(x_train_q, y_train, batch_size=64, epochs=15, validation_data=(x_test_q,y_test))

Epoch 1/15
938/938 [==============================] - 3186s 3s/step - loss: 0.7750 - auc: 0.9700 - accuracy: 0.7970 - recall: 0.6134 - precision: 0.9336 - val_loss: 0.4890 - val_auc: 0.9862 - val_accuracy: 0.8708 - val_recall: 0.8009 - val_precision: 0.9313
Epoch 2/15
938/938 [==============================] - 2323s 2s/step - loss: 0.4590 - auc: 0.9871 - accuracy: 0.8763 - recall: 0.8179 - precision: 0.9324 - val_loss: 0.4068 - val_auc: 0.9895 - val_accuracy: 0.8915 - val_recall: 0.8458 - val_precision: 0.9354
Epoch 3/15
938/938 [==============================] - 2085s 2s/step - loss: 0.4113 - auc: 0.9890 - accuracy: 0.8878 - recall: 0.8428 - precision: 0.9331 - val_loss: 0.3792 - val_auc: 0.9902 - val_accuracy: 0.8986 - val_recall: 0.8588 - val_precision: 0.9369
Epoch 4/15
938/938 [==============================] - 2202s 2s/step - loss: 0.3898 - auc: 0.9898 - accuracy: 0.8925 - recall: 0.8534 - precision: 0.9345 - val_loss: 0.3632 - val_auc: 0.9908 - val_accuracy: 0.9014 - val_recall: